# Lab 3.1: LangGraph Persistence (Banking Assistant)

In this lab, we will add **Persistence** (Memory) to our graph. This allows the graph to remember the state (e.g., account context) across different interactions, enabling long-running conversations.

## Key Concepts
1. **Checkpointer**: A mechanism to save the state of the graph at every step.
2. **MemorySaver**: An in-memory checkpointer for development/testing.
3. **Thread ID**: A unique identifier to separate different user sessions.

In [1]:
# 1. Install Dependencies
%pip install -qU langchain-groq langchain-community langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# 2. Setup API Keys
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

Enter your Groq API Key: ··········


### 3.1 Define State and Imports
First, we define the structure of our graph's state and import necessary libraries.

In [3]:
# Imports and State Definition
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage

class State(TypedDict):
    messages: Annotated[list, add_messages]

### 3.2 Initialize LLM and Persona
We set up the ChatGroq model and define the system prompt for the Banking Assistant.

In [4]:
# Initialize LLM
llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0,
    reasoning_format="parsed"
)

# Banking Assistant Persona
sys_msg = SystemMessage(content="""You are a helpful Wells Fargo banking assistant.
You can answer questions about account balances and transactions.
- If the user asks about a balance or transactions, ALWAYS check if they have specified the account type (Checking or Savings).
- If they haven't specified, ask them "Which account?".
- If they have specified (or it's clear from conversation history), provide a mock answer.
    - Checking Balance: $2,500
    - Savings Balance: $10,500
    - Checking Transactions: Walmart (-$50), Deposit (+$1000)
    - Savings Transactions: Interest (+$50)
""")

### 3.3 Define Node Function
Here we define the chatbot node. We add a print statement to see when this node is actually executed.

In [5]:
def chatbot(state: State):
    print("--- Node: Chatbot Activated ---")
    return {"messages": [llm.invoke([sys_msg] + state["messages"])]}

### 3.4 Verification: Test the Chatbot Node
Before building the graph, let's test the `chatbot` function directly to ensure it responds correctly.

In [6]:
# Verification: Test the node directly
print("Running verification...")
dummy_state = {"messages": [("user", "Hello, what can you do?")]}
response = chatbot(dummy_state)
print("Response:", response['messages'][0].content)

Running verification...
--- Node: Chatbot Activated ---
Response: Hello! I can help you check your account balances, view recent transactions, and provide details about your Checking or Savings accounts. For example, you can ask:  
- "What's my Checking balance?"  
- "Show my Savings transactions."  
Let me know how I can assist! 😊


### 3.5 Build the Graph
Now we assemble the nodes and edges into a `StateGraph`.

In [7]:
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

## 4. Add Persistence
We use `MemorySaver` to store the state.

In [8]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Compile with checkpointer
graph = graph_builder.compile(checkpointer=memory)

## 5. Run with Thread ID
We use `configurable` to specify the thread. In this initial interaction, we establish the context (Checking Account).

In [9]:
config = {"configurable": {"thread_id": "1"}}

# User establishes context
user_input = "I have a question about my Checking account."

for event in graph.stream({"messages": [("user", user_input)]}, config=config):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

--- Node: Chatbot Activated ---
Assistant: Your **Checking account** balance is **$2,500**.  

**Recent transactions:**  
- Walmart: -$50  
- Deposit: +$1,000  

Let me know if you'd like more details or assistance! 😊


## 6. Verify Memory (Persistence)
Now we ask "What is the balance?" **without specifying the account**. The bot should remember we are talking about **Checking** because of the shared `thread_id`.

In [10]:
user_input = "What is the balance?"

# Note: We are using the same 'config' with thread_id='1'
for event in graph.stream({"messages": [("user", user_input)]}, config=config):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

--- Node: Chatbot Activated ---
Assistant: Your **Checking account** balance is **$2,500**.  

**Recent activity:**  
- Walmart: -$50  
- Deposit: +$1,000  

Let me know if you need further assistance! 😊


## 7. New Thread (No Memory)
If we change the `thread_id` to "2", the memory should be empty. The bot should NOT know which account we are referring to.

In [11]:
config_new = {"configurable": {"thread_id": "2"}}

# Asking the same question in a new thread
user_input = "What is the balance?"

for event in graph.stream({"messages": [("user", user_input)]}, config=config_new):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

--- Node: Chatbot Activated ---
Assistant: Which account?


In [12]:
user_input = "For the Savings account."

# Note: We are using the same 'config' with thread_id='1'
for event in graph.stream({"messages": [("user", user_input)]}, config=config_new):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

--- Node: Chatbot Activated ---
Assistant: Your **Savings Account Balance** is **$10,500**.  

**Recent Savings Transactions:**  
- Interest: +$50  

Let me know if you need further details! 😊
